In [1]:
import os
os.chdir('..')

In [ ]:
from torchvision import datasets, transforms

In [ ]:
train_ds = datasets.CIFAR10('data/cifar10', download=True) 
#             transform=transforms.Compose(
#     [transforms.ToTensor(),
#      transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

In [ ]:
type(train_ds[1][1])

In [ ]:
help(tf.keras.utils.get_file)

In [21]:
import numpy as np
import tensorflow as tf
import h5py
from pathlib import Path
import os

def download_all_data():
    data_dir = Path('data/cifar100fed')

    path = tf.keras.utils.get_file(
          'fed_cifar100.tar.bz2',
          origin='https://storage.googleapis.com/tff-datasets-public/fed_cifar100.tar.bz2',
          file_hash='e8575e22c038ecef1ce6c7d492d7abee7da13b1e1ba9b70a7fc18531ba7590de',
          hash_algorithm='sha256',
          extract=True,
          archive_format='tar')
    dir_path = os.path.dirname(path)

    def download_split(data_set):
        split_dir = data_dir / data_set
        try:
            os.makedirs(split_dir)
        except FileExistsError:
            return
        h5 = h5py.File(os.path.join(dir_path, f'fed_cifar100_{data_set}.h5'), "r")["examples"]

        for client_id in h5.keys():
            images = []
            labels = []
            for item_id in range(100):
                images.append(h5[client_id]["image"][item_id])
                labels.append(h5[client_id]["label"][item_id])
            images = np.stack(images, axis=0)
            labels = np.stack(labels, axis=0)
            np.save(split_dir / f"{client_id}_img", images)
            np.save(split_dir / f"{client_id}_label", labels)
        
    download_split("train")
    download_split("test")

In [22]:
download_all_data()

In [35]:
import numpy as np
import tensorflow as tf
import h5py
from pathlib import Path
import os

def get_data(split):
    if isinstance(split, str) and split == "test":
        data_set = "test"
        client_ids = [str(i) for i in range(100)]
    else:
        data_set = "train"
        client_ids = split
        
    images = []
    labels = []
    for c in client_ids:
        i, l = get_client_data(data_set, c) 
        images.append(i)
        labels.append(l)
        
    return np.concatenate(images), np.concatenate(labels)

def get_client_data(data_set, client_id):
    data_dir = Path('data/cifar100fed')
    split_dir = data_dir / data_set

    images = np.load(split_dir / f"{client_id}_img.npy")
    labels = np.load(split_dir / f"{client_id}_label.npy")
    return images, labels

In [36]:
i, l = get_data(["2", "3"])

In [38]:
l.shape

(200,)

In [ ]:
def get_cifar100fed_h5(split):
    dir_path = os.path.dirname(get_cifar100fed_h5.path)
    return h5py.File(os.path.join(dir_path, f'fed_cifar100_{split}.h5'), "r")["examples"]

get_cifar100fed_h5.path = tf.keras.utils.get_file(
      'fed_cifar100.tar.bz2',
      origin='https://storage.googleapis.com/tff-datasets-public/fed_cifar100.tar.bz2',
      file_hash='e8575e22c038ecef1ce6c7d492d7abee7da13b1e1ba9b70a7fc18531ba7590de',
      hash_algorithm='sha256',
      extract=True,
      archive_format='tar')

In [ ]:
from torch.utils.data import Dataset

class CIFAR10Fed(Dataset):
    N_ELEMENTS_PER_CLIENT = 100
    def __init__(self, split, transform=None):
        if isinstance(split, str) and split == "test":
            self.h5 = get_cifar100fed_h5('test')
            self.clients = list(self.h5.keys())
        else:
            self.h5 = get_cifar100fed_h5('train')
            self.clients = split
        
        self.transform = transform
        
    def __len__(self):
        return len(self.clients) * self.N_ELEMENTS_PER_CLIENT
    
    def __getitem__(self, idx):
        client_id = idx // self.N_ELEMENTS_PER_CLIENT
        item_id = idx % self.N_ELEMENTS_PER_CLIENT
        
        client = self.h5[self.clients[client_id]]
        img = client["image"][item_id]
        label = client["label"][item_id]
        
        if self.transform is not None:
            img = self.transform(img)

        return img, label

In [ ]:
ds = CIFAR10Fed('test', transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

In [ ]:
ds[0]

In [ ]:
for k in h5.keys():
    assert len(h5["0"]["label"]) == 100

In [ ]:
import numpy as np
np.array_split(np.array(list(h5.keys())), 7)

In [ ]:
[str(x) for x in np.arange(500)]

In [ ]:
np.array(list(h5.keys()))

In [ ]:
for k, v in h5["examples"]["1"].items():
    print(k)

In [ ]:
h5["1"]["image"][0]